# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    team = cities['NHL'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team['Population']= cities['Population (2016 est.)[8]']
    team = pd.melt(team, id_vars = ['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)

    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w]*\ ','')

    _df = nhl_df[nhl_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'\*','')
    _df = _df[['team','W','L']]

    dropList=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            dropList.append(i)
    _df=_df.drop(dropList)
    _df['team']=_df['team'].str.replace('[\w.]*\ ','')

    _df = _df.astype({'team':str,'W':int,'L':int})
    _df['W/L%']= _df['W']/(_df['W']+_df['L'])

    merge = pd.merge(team,_df,'outer', on = 'team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})
    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [2]:
nhl_correlation()

<ipython-input-1-b0972728d750>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['team'] = _df['team'].str.replace(r'\*','')


0.012486162921209881

In [36]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]','')
cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]','')
cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]','')
cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]','')

#def nhl_correlation1(): 
team = cities['NHL'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
team['Metropolitan area']=cities['Metropolitan area']
team['Population']= cities['Population (2016 est.)[8]']
team = pd.melt(team, id_vars = ['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)

team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
team['team']=team['team'].str.replace('[\w]*\ ','')

nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df['team'] = nhl_df['team'].str.replace(r'\*','')
nhl_df = nhl_df[['team','W','L']]

dropList=[]
for i in range(nhl_df.shape[0]):
    row=nhl_df.iloc[i]
    if row['team']==row['W'] and row['L']==row['W']:
        dropList.append(i)
nhl_df=nhl_df.drop(dropList)
nhl_df['team']=nhl_df['team'].str.replace('[\w.]*\ ','')

nhl_df = nhl_df.astype({'team':str,'W':int,'L':int})
nhl_df['W/L%']= nhl_df['W']/(nhl_df['W']+nhl_df['L'])

merge = pd.merge(team,nhl_df,'outer', on = 'team')
merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})
population_by_region = merge['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.012486162921209881

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NBA'] = cities['NBA'].str.replace('\[.*\]','')

def nba_correlation():
    team = cities['NBA'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team['Population']= cities['Population (2016 est.)[8]']
    team = pd.melt(team, id_vars=['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})   
    team['team'] = team['team'].replace(r'\[.*\]','')
    
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NBA'] = cities['NBA'].str.replace('\[.*\]','')

team = cities['NBA'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
team['Metropolitan area'] = cities['Metropolitan area']
team['Population']= cities['Population (2016 est.)[8]']
team = pd.melt(team, id_vars=['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)
team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})   

nba_df = nba_df[nba_df['year']==2018]
nba_df = nba_df[['team','W/L%']]
nba_df['team'] = nba_df['team'].str.replace('\(\d*\)',"")
nba_df['team'] = nba_df['team'].str.replace('\*','')
nba_df['team'] = nba_df['team'].str.replace('\w* ','')
nba_df['team'] = nba_df['team'].str.strip()
nba_df=nba_df.astype({'team':str,'W/L%':float})

merge=pd.merge(team,nba_df,how='outer', on='team').dropna()
merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

population_by_region = merge['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)[0]

-0.17665034108234076

In [8]:
team

,Metropolitan area,Population,team
0,New York City,20153634,Knicks
1,Los Angeles,13310447,Lakers
2,San Francisco Bay Area,6657982,Warriors
3,Chicago,9512999,Bulls
4,Dallas–Fort Worth,7233323,Mavericks
5,"Washington, D.C.",6131977,Wizards
6,Boston,4794447,Celtics
7,Minneapolis–Saint Paul,3551036,Timberwolves
8,Denver,2853077,Nuggets
9,Miami–Fort Lauderdale,6066387,Heat


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [68]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['MLB'] = cities['MLB'].str.replace('\[.*\]','')

team = cities['MLB'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
team['Metropolitan area'] = cities['Metropolitan area']
team['Population']= cities['Population (2016 est.)[8]']
team = pd.melt(team, id_vars=['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)
team['team']=team['team'].str.replace('\ Sox','Sox')
team['team']=team['team'].str.extract('(?=[\w\ ]*)([A-Za-z]*$)')
team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})

mlb_df = mlb_df[mlb_df['year']==2018]
mlb_df = mlb_df[['team','W-L%']]
mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
mlb_df['team'] = mlb_df['team'].str.extract('(?=[\w\ ]*)([A-Za-z]*$)')
mlb_df=mlb_df.astype({'team':str,'W-L%':float})

merge=pd.merge(team,mlb_df,how='outer', on='team').dropna()
merge=merge.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Population': np.nanmean})

population_by_region = merge['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge['W-L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)

(0.15003737475409495, 0.46442827201123427)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [38]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

team = cities['NFL'].str.extract('([A-Z]?[a-z0-9]*\ [A-Z]?[a-z0-9]*|[A-Z]?[a-z0-9]*)([A-Z]?[a-z0-9]*\ [A-Z]?[a-z0-9]*|[A-Z]?[a-z0-9]*)')
team['Metropolitan area'] = cities['Metropolitan area']
team['Population']= cities['Population (2016 est.)[8]']
team = pd.melt(team, id_vars=['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)
team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})   

nfl_df = nfl_df[nfl_df['year']==2018]
nfl_df = nfl_df[['team','W-L%']]
nfl_df['team'] = nfl_df['team'].str.replace('[\*\+]','')
droplist = []
for i in range (nfl_df.shape[0]):
    row = nfl_df.iloc[i,:]
    if row['team'] == row['W-L%']:
        droplist.append(i)
nfl_df = nfl_df.drop(droplist)
nfl_df['team'] = nfl_df['team'].str.extract('(?=[\w\s]*)([A-Za-z0-9]*$)')

merge = pd.merge(team, nfl_df,how='outer', on='team').dropna()
merge = merge.astype({'Metropolitan area':str,'W-L%':float,'Population':float})
merge = merge.groupby('Metropolitan area').agg({'W-L%':np.nanmean,'Population':np.nanmean})

population_by_region = merge['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)



(0.00428214143639298, 0.9824114740736537)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [70]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

def NHL_df():
    team = cities['NHL'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team['Population']= cities['Population (2016 est.)[8]']
    team = pd.melt(team, id_vars = ['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)

    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w]*\ ','')

    NHL_df = nhl_df
    NHL_df = NHL_df[NHL_df['year'] == 2018]
    NHL_df['team'] = NHL_df['team'].str.replace(r'\*','')
    NHL_df = NHL_df[['team','W','L']]

    dropList=[]
    for i in range(NHL_df.shape[0]):
        row=NHL_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            dropList.append(i)
    NHL_df=NHL_df.drop(dropList)
    NHL_df['team']=NHL_df['team'].str.replace('[\w.]*\ ','')

    NHL_df = NHL_df.astype({'team':str,'W':int,'L':int})
    NHL_df['W/L%']= NHL_df['W']/(NHL_df['W']+NHL_df['L'])

    merge = pd.merge(team,NHL_df,'outer', on = 'team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})
    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    return merge[['W/L%']]

def NBA_df():
    team = cities['NBA'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team['Population']= cities['Population (2016 est.)[8]']
    team = pd.melt(team, id_vars=['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})   

    NBA_df = nba_df
    NBA_df = NBA_df[NBA_df['year']==2018]
    NBA_df = NBA_df[['team','W/L%']]
    NBA_df['team'] = NBA_df['team'].str.replace('\(\d*\)',"")
    NBA_df['team'] = NBA_df['team'].str.replace('\*','')
    NBA_df['team'] = NBA_df['team'].str.replace('\w* ','')
    NBA_df['team'] = NBA_df['team'].str.strip()
    NBA_df = NBA_df.astype({'team':str,'W/L%':float})

    merge=pd.merge(team,NBA_df,how='outer', on='team').dropna()
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]

def MLB_df():
    team = cities['MLB'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team['Population']= cities['Population (2016 est.)[8]']
    team = pd.melt(team, id_vars=['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)
    team['team']=team['team'].str.replace('\ Sox','Sox')
    team['team']=team['team'].str.extract('(?=[\w\ ]*)([A-Za-z]*$)')
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
    MLB_df = mlb_df
    MLB_df = MLB_df[MLB_df['year']==2018]
    MLB_df = MLB_df[['team','W-L%']]
    MLB_df['team'] = MLB_df['team'].str.replace('\ Sox','Sox')
    MLB_df['team'] = MLB_df['team'].str.extract('(?=[\w\ ]*)([A-Za-z]*$)')
    MLB_df.rename(columns={'W-L%':'W/L%'},inplace=True)
    MLB_df = MLB_df.astype({'team':str,'W/L%':float})

    merge=pd.merge(team,MLB_df,how='outer', on='team').dropna()
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    return merge[['W/L%']]

def NFL_df():
    team = cities['NFL'].str.extract('([A-Z]?[a-z0-9]*\ [A-Z]?[a-z0-9]*|[A-Z]?[a-z0-9]*)([A-Z]?[a-z0-9]*\ [A-Z]?[a-z0-9]*|[A-Z]?[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team['Population']= cities['Population (2016 est.)[8]']
    team = pd.melt(team, id_vars=['Metropolitan area','Population'],value_name='team').drop(columns=['variable']).replace("",np.nan).dropna().reset_index(drop=True)
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})   
    
    NFL_df = nfl_df
    NFL_df = NFL_df[NFL_df['year']==2018]
    NFL_df = NFL_df[['team','W-L%']]
    NFL_df.rename(columns={'W-L%':'W/L%'},inplace = True)
    NFL_df['team'] = NFL_df['team'].str.replace('[\*\+]','')
    droplist = []
    for i in range (NFL_df.shape[0]):
        row = NFL_df.iloc[i,:]
        if row['team'] == row['W/L%']:
            droplist.append(i)
    NFL_df = NFL_df.drop(droplist)
    NFL_df['team'] = NFL_df['team'].str.extract('(?=[\w\s]*)([A-Za-z0-9]*$)')
    
    merge = pd.merge(team, NFL_df,how='outer', on='team').dropna()
    merge = merge.astype({'Metropolitan area':str,'W/L%':float,'Population':float})
    merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})
    
    return merge[['W/L%']]

def create_df(sport):
    if sport =='NFL':
        return NFL_df()
    elif sport =='NBA':
        return NBA_df()
    elif sport =='NHL':
        return NHL_df()
    elif sport =='MLB':
        return MLB_df()
    
def sports_team_performance():
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in sports:
        for j in sports:
            if i!=j:
                a = pd.merge(create_df(i),create_df(j),how='inner',on = 'Metropolitan area')
                p_values.loc[i,j]=stats.ttest_rel(a['W/L%_x'],a['W/L%_y'])[1]

    return p_values

In [71]:
sports_team_performance()

<ipython-input-70-f6e546246e38>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NHL_df['team'] = NHL_df['team'].str.replace(r'\*','')
<ipython-input-70-f6e546246e38>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NHL_df['team'] = NHL_df['team'].str.replace(r'\*','')
<ipython-input-70-f6e546246e38>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

,NFL,NBA,NHL,MLB
NFL,NaN,0.876115,0.030318,0.803459
NBA,0.876115,NaN,0.019022,0.880018
NHL,0.030318,0.019022,NaN,0.000703
MLB,0.803459,0.880018,0.000703,NaN
